In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:
# Function to extract Product Title
def get_title(soup):
    try:
        title = soup.find("span", attrs={"id": 'productTitle'})
        title_string = title.text.strip()
    except AttributeError:
        title_string = ""
    return title_string

# Function to extract Product Price
def get_price(soup):
    try:
        price = soup.find("span", attrs={'id': 'priceblock_ourprice'}).string.strip()
    except AttributeError:
        try:
            price = soup.find("span", attrs={'id': 'priceblock_dealprice'}).string.strip()
        except AttributeError:
            price = ""
    return price

# Function to extract Product Rating
def get_rating(soup):
    try:
        rating = soup.find("i", attrs={'class': 'a-icon a-icon-star a-star-4-5'}).string.strip()
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class': 'a-icon-alt'}).string.strip()
        except AttributeError:
            rating = ""
    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id': 'acrCustomerReviewText'}).string.strip()
    except AttributeError:
        review_count = ""
    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id': 'availability'})
        available = available.find("span").string.strip()
    except AttributeError:
        available = "Not Available"
    return available

In [3]:

if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36 Edg/126.0.0.0', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.com/s?k=gaming+laptop&crid=1IS6RTSXEY564&sprefix=gammi%2Caps%2C613&ref=nb_sb_ss_w_hit-vc-lth_gaming-laptop_k1_2_5"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))

    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)


In [4]:
amazon_df

,title,price,rating,reviews,availability
2,Acer Nitro V Gaming Laptop | Intel Core i5-134...,,4.5 out of 5 stars,"4,489 ratings",In Stock
3,"ASUS ROG Strix G16 (2024) Gaming Laptop, 16” 1...",,4.4 out of 5 stars,"1,124 ratings",In Stock
4,"ACEMAGIC Gaming Laptop,Laptop with AMD Ryzen 7...",,5.0 out of 5 stars,6 ratings,In Stock
10,"HP Victus 15 Gaming Laptop, 15.6"" FHD 144Hz Di...",,4.3 out of 5 stars,707 ratings,Only 12 left in stock - order soon.
12,ACEMAGIC 2024 Newest Gaming Laptop with Backli...,,4.2 out of 5 stars,43 ratings,Only 18 left in stock - order soon.
13,Acer Nitro V Gaming Laptop | Intel Core i7-136...,,4.4 out of 5 stars,131 ratings,In Stock
14,"ASUS TUF Gaming A15 (2024) Gaming Laptop, 15.6...",,4.4 out of 5 stars,521 ratings,In Stock
15,"MSI Katana 15 15.6"" 144Hz FHD Gaming Laptop: 1...",,4.3 out of 5 stars,290 ratings,Only 1 left in stock - order soon.
17,MSI Thin 15 15.6” 144Hz FHD Gaming Laptop: Int...,,5.0 out of 5 stars,6 ratings,In Stock
18,"ASUS ROG Strix G16 (2024) Gaming Laptop, 16” 1...",,4.4 out of 5 stars,112 ratings,In Stock
